In [51]:
import json
import pandas
import numpy
import os
import sys

from datetime import date
from os import path

In [2]:
import json
import os
import pandas
import pyarrow

from dotenv import load_dotenv
from sqlalchemy import create_engine

# Steps to install
# 1. pip install sqlalchemy-bigquery google-cloud-bigquery-storage pyarrow
# 2. Copy the credentials file to wherever you set BIGQUERY_CREDENTIALS_PATH to

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')

In [3]:
sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
import create_queries
%load_ext autoreload
%autoreload 2

In [59]:
COMPANY_NAME = 'Royal Apothecary'
COMPANY_IDENTIFIER = 'RA'
TRANSFER_PACKAGES_START_DATE = '2020-01-01'
SALES_TRANSACTIONS_START_DATE = '2020-01-01'
INVENTORY_DATES = [
    '09/30/2020',
    '10/31/2020',
    '11/30/2020',
    '12/31/2020',
    '01/31/2021',
    '02/28/2021',
    '03/31/2021',
    '04/30/2021',
    '05/31/2021',
    '06/30/2021',
    '07/31/2021',
    '08/31/2021',
    '09/30/2021',
]
INVENTORY_DATE = '10/11/2021'
TODAY_DATE = date.today().strftime('%m/%d/%Y')

In [46]:
company_incoming_transfer_packages_query = create_queries.create_company_incoming_transfer_packages_query(COMPANY_IDENTIFIER, TRANSFER_PACKAGES_START_DATE)
company_outgoing_transfer_packages_query = create_queries.create_company_outgoing_transfer_packages_query(COMPANY_IDENTIFIER, TRANSFER_PACKAGES_START_DATE)
company_sales_transactions_query = create_queries.create_company_sales_transactions_query(COMPANY_IDENTIFIER, SALES_TRANSACTIONS_START_DATE)
company_inventory_packages_query = create_queries.create_company_inventory_packages_query(COMPANY_IDENTIFIER)

engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))
company_incoming_transfer_packages_dataframe = pandas.read_sql_query(company_incoming_transfer_packages_query, engine)
company_outgoing_transfer_packages_dataframe = pandas.read_sql_query(company_outgoing_transfer_packages_query, engine)
company_sales_transactions_dataframe = pandas.read_sql_query(company_sales_transactions_query, engine)
company_inventory_packages_dataframe = pandas.read_sql_query(company_inventory_packages_query, engine)

In [47]:
company_incoming_transfer_packages_dataframe

,package_row_id,delivery_type,license_number,manifest_number,created_date,received_datetime,shipper_facility_license_number,shipper_facility_name,recipient_facility_license_number,recipient_facility_name,...,product_category_name,product_name,package_lab_results_status,shipper_wholesale_price,shipped_quantity,shipped_unit_of_measure,received_quantity,received_unit_of_measure,item_unit_weight,item_unit_weight_unit_of_measure_name
0,5958adc8-baab-4a29-a1f1-5de454fefe71,INCOMING_FROM_VENDOR,C10-0000596-LIC,0002586291,2021-10-18,NaT,C11-0000256-LIC,"CALIFORNIA LOYAL, INC.",C10-0000596-LIC,ROYAL APOTHECARY L.L.C.,...,Vape Cartridge (weight - each),Bloom Live Resin 1G Bacio Driver Hybrid Carts ...,passed,270.00,12.0,Each,NaN,None,1.0,Grams
1,67ea3ed1-c5bb-42eb-bbf1-2db603771478,INCOMING_FROM_VENDOR,C10-0000596-LIC,0002586291,2021-10-18,NaT,C11-0000256-LIC,"CALIFORNIA LOYAL, INC.",C10-0000596-LIC,ROYAL APOTHECARY L.L.C.,...,Flower (packaged eighth - each),Bloom Sun Grown 27% Strawberry Fields Sativa B...,passed,240.00,24.0,Each,NaN,None,3.5,Grams
2,8f63dde8-e18d-40c6-b441-7c6b8cd54f1a,INCOMING_FROM_VENDOR,C10-0000596-LIC,0002586291,2021-10-18,NaT,C11-0000256-LIC,"CALIFORNIA LOYAL, INC.",C10-0000596-LIC,ROYAL APOTHECARY L.L.C.,...,Vape Cartridge (weight - each),Bloom Live Resin 0.5G Raskals Fire OG Hybrid C...,passed,0.12,12.0,Each,NaN,None,0.5,Grams
3,4bb8b9c5-729b-4187-a224-c2b20348e53c,INCOMING_FROM_VENDOR,C10-0000596-LIC,0002586291,2021-10-18,NaT,C11-0000256-LIC,"CALIFORNIA LOYAL, INC.",C10-0000596-LIC,ROYAL APOTHECARY L.L.C.,...,Flower (packaged half ounce - each),Bloom SGF Half Ounce Smalls 22% Vanilla Frosti...,passed,420.00,12.0,Each,NaN,None,0.5,Ounces
4,d5d6f9f4-6b4d-4d37-af86-b1b4295e865c,INCOMING_FROM_VENDOR,C10-0000596-LIC,0002586291,2021-10-18,NaT,C11-0000256-LIC,"CALIFORNIA LOYAL, INC.",C10-0000596-LIC,ROYAL APOTHECARY L.L.C.,...,Flower (packaged gram - each),MAADF02435 Bloom Sun Grown Flower 22.4% True O...,passed,240.00,24.0,Each,NaN,None,3.5,Grams
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5625,c9758e23-75fa-41cd-9650-37b11bd94f26,INCOMING_FROM_VENDOR,C10-0000596-LIC,0000317153,2020-02-12,2020-03-04 01:13:40+00:00,C12-0000175-LIC,CONNECTED TRANSPORTATION PARTNERS INC.,C10-0000596-LIC,ROYAL APOTHECARY L.L.C.,...,Extract (weight - each),831 Extracts Live Resin Sugar Gelato,failed,NaN,1.0,Each,1.0,Each,1.0,Grams
5626,cd5ae2d2-972f-44f4-a4c2-198d555002e2,INCOMING_FROM_VENDOR,C10-0000596-LIC,0000317153,2020-02-12,2020-03-04 01:13:40+00:00,C12-0000175-LIC,CONNECTED TRANSPORTATION PARTNERS INC.,C10-0000596-LIC,ROYAL APOTHECARY L.L.C.,...,Pre-Roll Infused,AQ Infused PreRoll,failed,NaN,25.0,Each,25.0,Each,1.0,Grams
5627,c6c58238-1701-485b-aed3-047603a361e1,INCOMING_FROM_VENDOR,C10-0000596-LIC,0000317153,2020-02-12,2020-03-04 01:13:40+00:00,C12-0000175-LIC,CONNECTED TRANSPORTATION PARTNERS INC.,C10-0000596-LIC,ROYAL APOTHECARY L.L.C.,...,Extract (weight - each),Acapulco Gold Vape Cart,failed,NaN,40.0,Each,40.0,Each,0.5,Grams
5628,71a388cf-de9d-40da-ad25-a7a8957f3345,INCOMING_FROM_VENDOR,C10-0000596-LIC,0000317153,2020-02-12,2020-03-04 01:13:40+00:00,C12-0000175-LIC,CONNECTED TRANSPORTATION PARTNERS INC.,C10-0000596-LIC,ROYAL APOTHECARY L.L.C.,...,Extract (volume - each),Acapulco Gold Vape Cart,failed,NaN,1.0,Each,1.0,Each,NaN,None


In [48]:
company_outgoing_transfer_packages_dataframe

,package_row_id,delivery_type,license_number,manifest_number,created_date,received_datetime,shipper_facility_license_number,shipper_facility_name,recipient_facility_license_number,recipient_facility_name,...,product_category_name,product_name,package_lab_results_status,shipper_wholesale_price,shipped_quantity,shipped_unit_of_measure,received_quantity,received_unit_of_measure,item_unit_weight,item_unit_weight_unit_of_measure_name
0,9dc3bcaf-af00-4cc2-aedc-fa5fc2f3e4d2,OUTGOING_TO_PAYOR,C10-0000596-LIC,0001647021,2021-03-05,2021-04-05 16:00:05+00:00,C10-0000596-LIC,ROYAL APOTHECARY L.L.C.,C10-0000481-LIC,"CALIFORNIA COMPASSIONATE CARE NETWORK, INC.",...,Vape Cartridge (weight - each),Lemon Berry Kush .5g Vape Cartridge,passed,NaN,1.0,Each,1.0,Each,0.5,Grams
1,38202e12-fd67-456d-9a7a-a4ee39357523,OUTGOING_TO_PAYOR,C10-0000596-LIC,0001647021,2021-03-05,2021-04-05 16:00:05+00:00,C10-0000596-LIC,ROYAL APOTHECARY L.L.C.,C10-0000481-LIC,"CALIFORNIA COMPASSIONATE CARE NETWORK, INC.",...,Vape Cartridge (weight - each),Lemon Jack .5g Vape Cartridge,passed,NaN,1.0,Each,1.0,Each,0.5,Grams
2,c02661d8-9f7f-4b7c-ac85-d320979565db,OUTGOING_TO_PAYOR,C10-0000596-LIC,0001647021,2021-03-05,2021-04-05 16:00:05+00:00,C10-0000596-LIC,ROYAL APOTHECARY L.L.C.,C10-0000481-LIC,"CALIFORNIA COMPASSIONATE CARE NETWORK, INC.",...,Vape Cartridge (weight - each),Star Fruit .5g Vape Cartridge,passed,NaN,1.0,Each,1.0,Each,0.5,Grams
3,93859204-f6e8-4bea-8714-1162e0112405,OUTGOING_TO_PAYOR,C10-0000596-LIC,0001613105,2021-02-25,2021-02-25 01:53:14+00:00,C10-0000596-LIC,ROYAL APOTHECARY L.L.C.,C9-0000309-LIC,WHOLE GREENS CALIFORNIA MANAGEMENT INC,...,Vape Cartridge (weight - each),Star Fruit .5g Vape Cartridge,passed,NaN,42.0,Each,42.0,Each,0.5,Grams
4,dd19088f-df01-4fae-97b8-015fbea926a2,OUTGOING_TO_PAYOR,C10-0000596-LIC,0001613105,2021-02-25,2021-02-25 01:53:14+00:00,C10-0000596-LIC,ROYAL APOTHECARY L.L.C.,C9-0000309-LIC,WHOLE GREENS CALIFORNIA MANAGEMENT INC,...,Vape Cartridge (weight - each),Lemon Berry Kush .5g Vape Cartridge,passed,NaN,42.0,Each,42.0,Each,0.5,Grams
5,dd548e2f-28e0-48b3-bd52-a1bc81ce8373,OUTGOING_TO_PAYOR,C10-0000596-LIC,0001613105,2021-02-25,2021-02-25 01:53:14+00:00,C10-0000596-LIC,ROYAL APOTHECARY L.L.C.,C9-0000309-LIC,WHOLE GREENS CALIFORNIA MANAGEMENT INC,...,Vape Cartridge (weight - each),Beary OG .5g Vape Cartridge,passed,NaN,42.0,Each,42.0,Each,0.5,Grams
6,83fda948-712c-4f7d-bf38-1b013f0f1fff,OUTGOING_TO_PAYOR,C10-0000596-LIC,0001602392,2021-02-23,2021-02-23 01:16:41+00:00,C10-0000596-LIC,ROYAL APOTHECARY L.L.C.,C11-0000137-LIC,FEELGOOD SALES GROUP,...,Vape Cartridge (weight - each),Lemon Berry Kush .5g Vape Cartridge,passed,NaN,600.0,Each,600.0,Each,0.5,Grams
7,8323d1f6-2bdf-416b-bc83-1afd006d3267,OUTGOING_TO_PAYOR,C10-0000596-LIC,0001310460,2020-12-03,2021-01-07 19:45:14+00:00,C10-0000596-LIC,ROYAL APOTHECARY L.L.C.,C11-0000171-LIC,"Shield Management Group, LLC",...,Pre-Roll Flower,"Gas Pack – 1g Cured Pre Roll 3 Pack - SFV OG, ...",passed,NaN,15.0,Each,15.0,Each,3.0,Grams
8,858b9b3d-4239-408c-b133-857660d2cf92,OUTGOING_TO_PAYOR,C10-0000596-LIC,0001221976,2020-11-10,2021-06-29 16:13:45+00:00,C10-0000596-LIC,ROYAL APOTHECARY L.L.C.,C11-0000349-LIC,MISSION HILLS PATIENTS COLLECTIVE,...,Flower,Cali Flwr Farm - Cali Gas (3.5 GM Flower) (#29...,passed,8.0,100.0,Grams,100.0,Grams,NaN,None
9,e5db0764-61ad-44bf-824d-a8733090640f,OUTGOING_TO_PAYOR,C10-0000596-LIC,0001105412,2020-10-09,2020-11-19 22:34:00+00:00,C10-0000596-LIC,ROYAL APOTHECARY L.L.C.,C11-0000578-LIC,"9701 ENTERPRISES, INC.",...,Pre-Roll Flower,Sticks Preroll - Chocolate Fire - 1G / Case of 44,passed,NaN,132.0,Each,132.0,Each,1.0,Grams


In [49]:
company_sales_transactions_dataframe

,id,license_number,receipt_number,rt_type,sales_customer_type,sales_datetime,total_packages,rt_total_price,tx_type,tx_package_id,tx_package_label,tx_product_name,tx_product_category_name,tx_unit_of_measure,tx_quantity_sold,tx_total_price
0,e437f5d8-a152-442b-a6aa-6285f577feab,C10-0000596-LIC,0137267538,active,Consumer,2021-10-18 20:59:49.260000+00:00,1,8.0,active,18234328,1A4060300003D59000020512,Viva Sativa - Flor Azul (3.5 GM Flower (Each))...,Flower (packaged eighth - each),Each,1.0,8.0
1,4c113b88-aad9-4dbd-a7d6-700cf4d8f80b,C10-0000596-LIC,0137265376,active,Consumer,2021-10-18 20:47:06.860000+00:00,1,13.5,active,18684123,1A4060300003D59000021083,Rich & Ruthless - CPT Glue (SMALLS) (3.5 GM Fl...,Flower (packaged eighth - each),Each,1.0,13.5
2,a19dc9d7-a044-409d-897b-c93ca8290714,C10-0000596-LIC,0137259302,active,Consumer,2021-10-18 20:16:08.830000+00:00,1,25.2,active,18459735,1A406030000339A000013005,Wedding Crashers 1g,Flower (packaged gram - each),Each,2.0,25.2
3,8500ea26-b645-4071-bc77-62b1191c02f8,C10-0000596-LIC,0137258305,active,Consumer,2021-10-18 20:11:21.280000+00:00,1,80.0,active,18576363,1A406030002E7C2000000081,Mother Tree - Apple Fritter,Flower (packaged eighth - each),Each,2.0,80.0
4,97e0150d-4095-428b-9107-8eeb2eec2e57,C10-0000596-LIC,0137258191,active,Consumer,2021-10-18 20:10:36.790000+00:00,1,22.5,active,18498741,1A4060300007D04000010646,Green Hornet - 2:1 - Good Night Grape - 10 Pack,Edible (weight - each),Each,1.0,22.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153158,9ccc7566-b22d-4381-88fe-607b2ac6c5f1,C10-0000596-LIC,0015284616,active,Consumer,2020-03-08 12:30:35.440000+00:00,3,117.0,active,3050501,1A406030000A3CF000000232,Clonbar King Louis 3.5g,Flower (packaged eighth - each),Each,2.0,63.0
153159,9ccc7566-b22d-4381-88fe-607b2ac6c5f1,C10-0000596-LIC,0015284616,active,Consumer,2020-03-08 12:30:35.440000+00:00,3,117.0,active,2634052,1A4060300007B40000003102,Cinn Indica 10 Pk,Edible (weight - each),Each,1.0,0.0
153160,9ccc7566-b22d-4381-88fe-607b2ac6c5f1,C10-0000596-LIC,0015284616,active,Consumer,2020-03-08 12:30:35.440000+00:00,3,117.0,active,3016876,1A4060300008D22000021564,STIIIZY - OG Kush Pod - 0.5g,Vape Cartridge (volume - each),Each,2.0,54.0
153161,9ccc7566-b22d-4381-88fe-607b2ac6c5f1,C10-0000596-LIC,0015284616,active,Consumer,2020-03-08 12:30:35.440000+00:00,3,117.0,active,2634052,1A4060300007B40000003102,Cinn Indica 10 Pk,Edible (weight - each),Each,1.0,0.0


In [50]:
company_inventory_packages_dataframe

,license_number,package_id,package_label,type,packaged_date,package_type,product_name,product_category_name,quantity,unit_of_measure,is_testing_sample,is_trade_sample,is_on_hold,archived_date,finished_date
0,C10-0000596-LIC,18809619,1A406030000390D000022366,active,2021-10-18,Product,Bloom Live Resin 0.5G Raskals Fire OG Hybrid C...,Vape Cartridge (weight - each),12.0,Each,False,False,False,None,None
1,C10-0000596-LIC,18809684,1A406030000390D000022372,active,2021-10-18,Product,MAADF03143 Bloom Sun Grown Flower GG4 Hybrid 1...,Flower (packaged eighth - each),24.0,Each,False,False,False,None,None
2,C10-0000596-LIC,18809681,1A406030000390D000022371,active,2021-10-18,Product,MAADF02435 Bloom Sun Grown Flower 22.4% True O...,Flower (packaged gram - each),24.0,Each,False,False,False,None,None
3,C10-0000596-LIC,18810070,1A406030000390D000022370,active,2021-10-18,Product,Bloom Sun Grown 27% Strawberry Fields Sativa B...,Flower (packaged eighth - each),24.0,Each,False,False,False,None,None
4,C10-0000596-LIC,18809622,1A406030000390D000022367,active,2021-10-18,Product,Bloom Live Resin 1G Ghost Train Haze Sativa Ca...,Vape Cartridge (weight - each),12.0,Each,False,False,False,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3420,C10-0000596-LIC,2579124,1A406030000A3CF000000146,active,2020-02-11,Product,THC D- XJ 13 Classic J’s Tin 3.5g,Pre-Roll Flower,27.0,Each,False,False,False,None,None
3421,C10-0000596-LIC,2579028,1A406030000A3CF000000147,active,2020-02-11,Product,THC D- Nerdz 3.5g Jar,Flower,14.0,Grams,False,False,False,None,None
3422,C10-0000596-LIC,2555191,1A40603000067F9000003872,active,2020-02-10,Product,THCD Estate Eighth Jar Indica Dosidos 3.5g,Flower,3.5,Grams,False,False,False,None,None
3423,C10-0000596-LIC,2555366,1A40603000067F9000003874,active,2020-02-10,Product,THCD Classic Js Tins Sativa XJ-13 3.5g,Pre-Roll Flower,23.0,Each,False,False,False,None,None


In [26]:
sys.path.append(path.realpath(path.join(os.getcwd(), "../../scripts/analysis")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))

from util import active_inventory_util as util

In [27]:
d = util.Download()
d.download_dataframes(
    incoming_transfer_packages_dataframe=company_incoming_transfer_packages_dataframe,
    outgoing_transfer_packages_dataframe=company_outgoing_transfer_packages_dataframe,
    sales_transactions_dataframe=company_sales_transactions_dataframe,
)

In [28]:
q = util.Query()
q.inventory_dates = INVENTORY_DATES
q.company_name = COMPANY_NAME

id_to_history = util.get_histories(d)
util.print_counts(id_to_history)
util.create_inventory_xlsx(id_to_history, q)

Only outgoing: 7
Only incoming: 1534
In and out: 8
In and sold at least once 4049
In and sold many times 3913
Total pkgs: 5630
Wrote result to out/HPCC_inventory_by_month.xls
Excluded 80 / 5630 packages from consideration (1.42%)
  MANY_INCOMING: 40 times
  MISSING_INCOMING: 40 times


In [33]:
date_to_inventory_packages_dataframe = {}

for inventory_date in INVENTORY_DATES:
    computed_inventory_package_records = util.create_inventory_dataframe_by_date(id_to_history, inventory_date)
    computed_inventory_packages_dataframe = pandas.DataFrame(
        computed_inventory_package_records,
        columns=[
            'package_id',
            'Arrived Date',
            'Product Category',
            'Product Name',
            'Current Quantity',
            'Sold Date',
        ]
    )
    date_to_inventory_packages_dataframe[inventory_date] = computed_inventory_packages_dataframe

09/30/2020 919
10/31/2020 1114
11/30/2020 1021
12/31/2020 1190
01/31/2021 1430
02/28/2021 1598
03/31/2021 1763
04/30/2021 1898
05/31/2021 2031
06/30/2021 2215
07/31/2021 2263
08/31/2021 2375
09/30/2021 2472


In [37]:
date_to_inventory_packages_dataframe.keys()

dict_keys(['09/30/2020', '10/31/2020', '11/30/2020', '12/31/2020', '01/31/2021', '02/28/2021', '03/31/2021', '04/30/2021', '05/31/2021', '06/30/2021', '07/31/2021', '08/31/2021', '09/30/2021'])

In [38]:
incoming_transfer_packages_dataframe = company_incoming_transfer_packages_dataframe[[
#     'date_type',
#     'transfer_row_id',
#     'delivery_row_id',
#     'package_row_id',
    'delivery_type',
    'manifest_number',
    'created_date',
    'received_datetime',
    'shipper_facility_license_number',
    'shipper_facility_name',
    'recipient_facility_license_number',
    'recipient_facility_name',
    'shipment_type_name',
    'shipment_transaction_type',
    'package_id',
    'package_label',
    'type',
    'product_category_name',
    'product_name',
    'shipper_wholesale_price',
    'shipped_quantity',
    'package_lab_results_status',
    'shipment_package_state',
    'is_testing_sample',
    'is_trade_sample'
]]
incoming_transfer_packages_dataframe

,delivery_type,manifest_number,created_date,received_datetime,shipper_facility_license_number,shipper_facility_name,recipient_facility_license_number,recipient_facility_name,shipment_type_name,shipment_transaction_type,...,package_label,type,product_category_name,product_name,shipper_wholesale_price,shipped_quantity,package_lab_results_status,shipment_package_state,is_testing_sample,is_trade_sample
0,INCOMING_FROM_VENDOR,0002586291,2021-10-18,NaT,C11-0000256-LIC,"CALIFORNIA LOYAL, INC.",C10-0000596-LIC,ROYAL APOTHECARY L.L.C.,Wholesale Manifest,Wholesale,...,1A406030000390D000022369,transfer,Vape Cartridge (weight - each),Bloom Live Resin 1G Bacio Driver Hybrid Carts ...,270.00,12.0,passed,Shipped,False,False
1,INCOMING_FROM_VENDOR,0002586291,2021-10-18,NaT,C11-0000256-LIC,"CALIFORNIA LOYAL, INC.",C10-0000596-LIC,ROYAL APOTHECARY L.L.C.,Wholesale Manifest,Wholesale,...,1A406030000390D000022370,transfer,Flower (packaged eighth - each),Bloom Sun Grown 27% Strawberry Fields Sativa B...,240.00,24.0,passed,Shipped,False,False
2,INCOMING_FROM_VENDOR,0002586291,2021-10-18,NaT,C11-0000256-LIC,"CALIFORNIA LOYAL, INC.",C10-0000596-LIC,ROYAL APOTHECARY L.L.C.,Wholesale Manifest,Wholesale,...,1A406030000390D000022366,transfer,Vape Cartridge (weight - each),Bloom Live Resin 0.5G Raskals Fire OG Hybrid C...,0.12,12.0,passed,Shipped,False,False
3,INCOMING_FROM_VENDOR,0002586291,2021-10-18,NaT,C11-0000256-LIC,"CALIFORNIA LOYAL, INC.",C10-0000596-LIC,ROYAL APOTHECARY L.L.C.,Wholesale Manifest,Wholesale,...,1A406030000390D000022365,transfer,Flower (packaged half ounce - each),Bloom SGF Half Ounce Smalls 22% Vanilla Frosti...,420.00,12.0,passed,Shipped,False,False
4,INCOMING_FROM_VENDOR,0002586291,2021-10-18,NaT,C11-0000256-LIC,"CALIFORNIA LOYAL, INC.",C10-0000596-LIC,ROYAL APOTHECARY L.L.C.,Wholesale Manifest,Wholesale,...,1A406030000390D000022371,transfer,Flower (packaged gram - each),MAADF02435 Bloom Sun Grown Flower 22.4% True O...,240.00,24.0,passed,Shipped,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5625,INCOMING_FROM_VENDOR,0000317153,2020-02-12,2020-03-04 01:13:40+00:00,C12-0000175-LIC,CONNECTED TRANSPORTATION PARTNERS INC.,C10-0000596-LIC,ROYAL APOTHECARY L.L.C.,Transfer,Standard,...,1A406030000A3CF000000203,transfer_incoming,Extract (weight - each),831 Extracts Live Resin Sugar Gelato,NaN,1.0,failed,Accepted,False,False
5626,INCOMING_FROM_VENDOR,0000317153,2020-02-12,2020-03-04 01:13:40+00:00,C12-0000175-LIC,CONNECTED TRANSPORTATION PARTNERS INC.,C10-0000596-LIC,ROYAL APOTHECARY L.L.C.,Transfer,Standard,...,1A4060300005C36000000076,transfer_incoming,Pre-Roll Infused,AQ Infused PreRoll,NaN,25.0,failed,Accepted,False,False
5627,INCOMING_FROM_VENDOR,0000317153,2020-02-12,2020-03-04 01:13:40+00:00,C12-0000175-LIC,CONNECTED TRANSPORTATION PARTNERS INC.,C10-0000596-LIC,ROYAL APOTHECARY L.L.C.,Transfer,Standard,...,1A40603000091B5000000467,transfer_incoming,Extract (weight - each),Acapulco Gold Vape Cart,NaN,40.0,failed,Accepted,False,False
5628,INCOMING_FROM_VENDOR,0000317153,2020-02-12,2020-03-04 01:13:40+00:00,C12-0000175-LIC,CONNECTED TRANSPORTATION PARTNERS INC.,C10-0000596-LIC,ROYAL APOTHECARY L.L.C.,Transfer,Standard,...,1A406030000A3CF000000201,transfer_incoming,Extract (volume - each),Acapulco Gold Vape Cart,NaN,1.0,failed,Accepted,False,False


In [41]:
import math

for inventory_date, inventory_packages_dataframe in date_to_inventory_packages_dataframe.items():
    print(inventory_date)
    print(f'# of packages in inventory: {len(inventory_packages_dataframe.index)}')

    inventory_with_incoming_transfer_packages_dataframe = inventory_packages_dataframe \
        .astype({'package_id': 'int64'}) \
        .merge(
            incoming_transfer_packages_dataframe.astype({'package_id': 'int64'}),
            on='package_id',
            how='inner',
            suffixes=('_l', '_r')
        )
    #     print(f'# of packages in inventory with incoming package: {len(inventory_with_incoming_transfer_packages_dataframe.index)}')

    inventory_with_cost_records = inventory_with_incoming_transfer_packages_dataframe.to_dict('record')

    total_valuation_cost = 0

    for inventory_with_cost_record in inventory_with_cost_records:
        incoming_shipped_price = inventory_with_cost_record['shipper_wholesale_price']
        if math.isnan(incoming_shipped_price):
            incoming_shipped_price = 0
        incoming_shipped_quantity = inventory_with_cost_record['shipped_quantity']
        current_quantity = inventory_with_cost_record['Current Quantity']
        total_valuation_cost += float(current_quantity) * (incoming_shipped_price / incoming_shipped_quantity)

    print(f'Inventory valuation (based on COST): ${total_valuation_cost}')
    print('')

09/30/2020
# of packages in inventory: 919
Inventory valuation (based on COST): $108816.45453254889

10/31/2020
# of packages in inventory: 1114
Inventory valuation (based on COST): $145435.74730180847

11/30/2020
# of packages in inventory: 1021
Inventory valuation (based on COST): $81077.23779955569

12/31/2020
# of packages in inventory: 1190
Inventory valuation (based on COST): $97124.63086885298

01/31/2021
# of packages in inventory: 1430
Inventory valuation (based on COST): $122057.34758210585

02/28/2021
# of packages in inventory: 1598
Inventory valuation (based on COST): $115113.92536990347

03/31/2021
# of packages in inventory: 1763
Inventory valuation (based on COST): $131913.43333389104

04/30/2021
# of packages in inventory: 1898
Inventory valuation (based on COST): $166593.58704512744

05/31/2021
# of packages in inventory: 2031
Inventory valuation (based on COST): $175601.77576315537

06/30/2021
# of packages in inventory: 2215
Inventory valuation (based on COST): $173

In [60]:
today_inventory_package_records = util.create_inventory_dataframe_by_date(id_to_history, TODAY_DATE)
today_inventory_packages_dataframe = pandas.DataFrame(
    today_inventory_package_records,
    columns=[
        'package_id',
        'arrived_date',
        'product_category_name',
        'product_name',
        'current_quantity',
        'sold_date',
    ]
)

In [66]:
today_inventory_packages_dataframe.sort_values('package_id')

,package_id,arrived_date,product_category_name,product_name,current_quantity,sold_date
1550,10020010,12/11/2020,Flower (packaged eighth - each),GMO Cookies - NGW 1/8th,1,
1554,10020095,12/11/2020,Other Concentrate (weight - each),Fire OG - NGW Diamonds 1g,2,
1555,10020096,12/11/2020,Other Concentrate (weight - each),Fire OG - NGW - Diamonds 1g,1,
1557,10020177,12/11/2020,Flower (packaged eighth - each),Slurricane - NGW 1/8th,1,
1551,10020190,12/11/2020,Flower (packaged eighth - each),Ice Cream Cake - NGW 1/8th,1,
...,...,...,...,...,...,...
1563,9973118,12/10/2020,Extract (weight - each),Alien OG Badder (Gram Jar),7,
1565,9973120,12/10/2020,Extract (weight - each),Fruit Smoothie Badder (Gram Jar),1,
1564,9973121,12/10/2020,Extract (weight - each),Wedding Cake Badder (Gram Jar),2,
1562,9973122,12/10/2020,Extract (weight - each),Wild Berry Sugar (Gram Jar),1,


In [65]:
company_inventory_packages_dataframe[[
    'package_id',
    'packaged_date',
    'product_category_name',
    'product_name',
    'quantity',
]].sort_values('package_id')

,package_id,packaged_date,product_category_name,product_name,quantity
1905,10020009,2020-12-14,Flower (packaged eighth - each),GMO Cookies - NGW 1/8th,1.0
1904,10020010,2020-12-14,Flower (packaged eighth - each),GMO Cookies - NGW 1/8th,1.0
1897,10020096,2020-12-14,Other Concentrate (weight - each),Fire OG - NGW - Diamonds 1g,1.0
1903,10020177,2020-12-14,Flower (packaged eighth - each),Slurricane - NGW 1/8th,1.0
1899,10020190,2020-12-14,Flower (packaged eighth - each),Ice Cream Cake - NGW 1/8th,1.0
...,...,...,...,...,...
2133,9929434,2020-10-20,Flower (packaged eighth - each),Wifi 8th,1.0
1911,9950719,2020-12-09,Flower (packaged eighth - each),MA201002-1 | COTC | PrePacked | Mandarin Cooki...,1.0
1912,9950905,2020-12-09,Flower (packaged eighth - each),JA201117-1 | COTC | PrePacked | Apple Jack | 3...,1.0
1909,9973122,2020-12-09,Extract (weight - each),Wild Berry Sugar (Gram Jar),1.0
